In [ ]:
import requests
import mysql.connector
import time
import discord_notifier

def db_connect():
    return mysql.connector.connect(user='lol_user', password='your_password',
                                   host='35.203.46.85', database='lol_retention')

def fetch_match_details(match_id):
    
    api_key = "RGAPI-9b8cc388-16ad-44a6-a6b3-5575e0535855"
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match details for {match_id}: {response.status_code}")
        return None

def insert_match_details(match_details):
    cnx = db_connect()
    cursor = cnx.cursor()
    try:
        add_match = """
        INSERT INTO match_details (
            match_id, game_creation, game_duration, game_mode, game_type, game_version, map_id, participant_id, summoner_id,
            puuid, champion_id, spell1_id, spell2_id, item0, item1, item2, item3, item4, item5, item6,
            kills, deaths, assists, total_damage_dealt, total_damage_taken, gold_earned, vision_score,
            total_minions_killed, team_position, win, main_puuid)
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            game_creation = VALUES(game_creation), game_duration = VALUES(game_duration),
            total_damage_dealt = VALUES(total_damage_dealt), total_damage_taken = VALUES(total_damage_taken),
            gold_earned = VALUES(gold_earned), vision_score = VALUES(vision_score)
        """
        cursor.executemany(add_match, match_details)
        cnx.commit()
        print(f"Inserted {len(match_details)} match details.")
    except mysql.connector.Error as err:
        print("Error:", err)
    finally:
        cursor.close()
        cnx.close()

def pull_and_insert_match_details():
    cnx = db_connect()
    cursor = cnx.cursor()
    query = """
    SELECT m.match_id, m.puuid AS main_puuid
    FROM match_ids m
    LEFT JOIN match_details md ON m.match_id = md.match_id
    WHERE md.match_id IS NULL
    """
    cursor.execute(query)
    match_ids = cursor.fetchall()
    cursor.close()
    cnx.close()

    batch_size = 100  # manage batch size
    accumulated_details = []
    for match_id, main_puuid in match_ids:
        time.sleep(1.3)  # manage API rate limits
        match_data = fetch_match_details(match_id)
        if match_data:
            for participant in match_data['info']['participants']:
                match_detail = (
                    match_id,
                    match_data['info']['gameCreation'],
                    match_data['info']['gameDuration'],
                    match_data['info']['gameMode'],
                    match_data['info']['gameType'],
                    match_data['info']['gameVersion'],
                    match_data['info']['mapId'],
                    participant.get('participantId'),
                    participant.get('summonerId'),
                    participant.get('puuid'),
                    participant.get('championId'),
                    participant.get('spell1Id'),
                    participant.get('spell2Id'),
                    participant.get('item0'),
                    participant.get('item1'),
                    participant.get('item2'),
                    participant.get('item3'),
                    participant.get('item4'),
                    participant.get('item5'),
                    participant.get('item6'),
                    participant.get('kills'),
                    participant.get('deaths'),
                    participant.get('assists'),
                    participant.get('totalDamageDealt'),
                    participant.get('totalDamageTaken'),
                    participant.get('goldEarned'),
                    participant.get('visionScore'),
                    participant.get('totalMinionsKilled'),
                    participant.get('teamPosition', 'UNKNOWN'),
                    participant.get('win'),
                    main_puuid
                )
                accumulated_details.append(match_detail)
                if len(accumulated_details) >= batch_size:
                    insert_match_details(accumulated_details)
                    accumulated_details = []  # Reset after batch insert

    # Insert any remaining details not reaching the batch size
    if accumulated_details:
        insert_match_details(accumulated_details)

if __name__ == "__main__":
    try: 
        pull_and_insert_match_details()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally: 
        discord_notifier.send_discord_message()